In [1]:
# import libraries
import pandas as pd
import numpy as np 
import warnings
import gmplot
import sys
import geopy
from geopy import distance
from copy import deepcopy
sys.path.append("../HistoricalData/")
from getData import get_data

warnings.filterwarnings('ignore')

In [2]:
# this call to get_data function that will take bounding box and timeframe and return cleaned data

UP_LEFT = (38.008050, -122.536985)    # (lat, lon)
UP_RIGHT = (38.008050, -122.186437)   # (lat, lon)
DOWN_RIGHT = (37.701933, -122.186437) # (lat, lon)
DOWN_LEFT = (37.701933, -122.536985)  # (lat, lon)
START_DATE = '2019/09/01' # begin date to start taking data
END_DATE = '2019/09/10'   # end date to start taking data
START_HOUR = '0'         # hour EACH DAY to start, this allows us to control for time of day effects
END_HOUR = '1'           # hour EACH DAY to end, this allows us to control for time of day effects
color = "red"

data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR)

In [ ]:
data_df.columns

In [ ]:
data_df.groupby(data_df.sensor_id).count()

In [ ]:
data_df.groupby(data_df.created).count()

In [30]:
class Chain:
    """ 
    Chain is a class that defines a chain of sensors in time and space. 
    A chain has a list of timestamps that are paired with a list of particle locations, intersecting sensors, and readings
    """
    def __init__(self, sensor_id, start_time):
        self.start_time = start_time
        self.time = [self.start_time]
        self.sensor_path = [sensor_id]
        self.last_sensor = sensor_id
        self.particle_path = [(data_df[data_df.sensor_id == sensor_id].lat.iloc[0], data_df[data_df.sensor_id == sensor_id].lon.iloc[0])]
        self.readings = [data_df[(data_df.sensor_id == sensor_id) & (data_df.created == start_time)]['2_5um'].iloc[0]]
    
    def plot_next(self):
        # update time
        MINUTES = 10 # Joined data seems to be in increments of 10 minutes
        origin_time = self.time[-1]
        destination_time = self.time[-1] + MINUTES
        self.time.append(destination_time)
        
        # need to take wind speed in knots
        origin = geopy.Point(self.particle_path[-1][0], self.particle_path[-1][1])
        km_1_knot_per_hour = 1.852
        km_hour_test = km_1_knot_per_hour * data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_speed
        if len(km_hour_test) == 0: # handle empty data
            return False
        km_hour = km_hour_test.iloc[0] # so you are grabbing from last sensor seen... might not be nearest wind data
        d = (MINUTES / 60) * km_hour 
    
        # convert wind_direction to a bearing
        bearing = data_df[(data_df.sensor_id == self.last_sensor) & (data_df.created == origin_time)].wind_direction.iloc[0] 
        try:
            b = int(bearing) + 180
        except: # 'VRB' = variable winds
            return False 
        if b >= 360:
            b -= 360
        
        # given: lat1, lon1, b = bearing in degrees, d = distance in kilometers
        if d == 0:
            destination = self.particle_path[-1]
        else:
            destination = geopy.distance.VincentyDistance(kilometers=d).destination(origin, b)
            destination = (destination.latitude, destination.longitude)
        
            # end chain if outside bounding box
            if (destination[0] > UP_LEFT[0]) or (destination[0] < DOWN_LEFT[0]) or (destination[1] < UP_LEFT[1]) or (destination[1] > DOWN_RIGHT[1]):
                return False
        
            self.particle_path.append(destination)
        
            # check is there's a sensor nearby
            CLOSE_ENOUGH_DISTANCE = 1 # have to be within 1000 meters
        
            closest_sensor_id = None
            closest_sensor_dist = CLOSE_ENOUGH_DISTANCE
            for sensor in data_df.sensor_id.unique(): # iterate through all sensors in bounding box
                test_sensor = (float(data_df[data_df.sensor_id == sensor].lat.iloc[0]), float(data_df[data_df.sensor_id == sensor].lon.iloc[0]))
                testy = geopy.point.Point(test_sensor)
                desty = geopy.point.Point(destination)
                proximity = geopy.distance.distance(testy, desty).km
                if (proximity < closest_sensor_dist):
                    closest_sensor_id = sensor
                    closest_sensor_dist = proximity
                
            # update other lists
            self.sensor_path.append(closest_sensor_id)
            if closest_sensor_id:
                self.last_sensor = closest_sensor_id
                try: # handle empty reading
                    self.readings.append(data_df[(data_df.sensor_id == closest_sensor_id) & (data_df.created == destination_time)]['2_5um'].iloc[0])
                except:
                    return False
        return True
    
    def plot_pollution(self):
        lats = []
        lons = []
        colors= ['white'] # first color not used
        link = 0 # counts the links
        reading = 0 # counts the readings
        for point in self.particle_path:
            lats.append(point[0])
            lons.append(point[1])
            if (link > 0) and (self.sensor_path[link]):
                reading += 1
                if self.readings[reading] > self.readings[reading-1]: # possible pollution source
                    colors.append('red')
                    look_back = link-1
                    while ((look_back >= 0) and (self.sensor_path[look_back] == None)):
                        colors.append('red')
                        look_back -= 1
                else: # no pollution source
                    colors.append('blue')
                    look_back = link-1
                    while ((look_back >= 0) and (self.sensor_path[look_back] == None)):
                        colors.append('blue')
                        look_back -= 1
            link += 1

        # fill in black color for chain links at end that never feed into another sensor
        if len(self.particle_path) > 1:
            for index in range(len(self.particle_path),-1,-1):
                try:
                    if self.sensor_path[index] == None:
                        colors.append('black')
                    else:
                        return lats, lons, colors
                except:
                    colors.append('black')
        return lats, lons, colors

In [31]:
chains = [] # a list of chain 

set_size_chain = 10
start_time = 201909030000

# build the chains
for sensor in data_df.sensor_id.unique(): # iterate through all sensors in bounding box
    chain = Chain(sensor, start_time) # init one chain per sensor
    time_step = 0
    while (chain.plot_next() & (time_step < set_size_chain)): # method returns false if out of bounds/data
        time_step += 1
    chains.append(chain) # add the chain object to the list of chains
    print("Chain", len(chains), "is", chain.sensor_path)
    print("Chain", len(chains), "readings are", chain.readings)
print("Done.")

Chain 1 is ['16939', '16943', None, None, None, None]
Chain 1 readings are [4.93, 7.61]
Chain 2 is ['16940', '16943', None, None, None, None]
Chain 2 readings are [3.5, 7.61]


KeyboardInterrupt: 

In [ ]:
# plot it
latitude_list = []
longitude_list = []
for row in range(len(data_df)):
    try:
        latitude_list.append(data_df.loc[row].lat)
        longitude_list.append(data_df.loc[row].lon)
    except:
        pass
gmap3 = gmplot.GoogleMapPlotter((max(latitude_list)+min(latitude_list))/2,
                                (max(longitude_list)+min(longitude_list))/2,
                                zoom=11)
gmap3.scatter(latitude_list, longitude_list, '# FF0000', 
                              size = 40, marker = False ) 

for chain in chains:
    lats, lons, colors = chain.plot_pollution()
    for index in range(1,len(lats)):
        gmap3.plot([lats[index-1], lats[index]], [lons[index-1], lons[index]], colors[index], edge_width = 2.5)
    
# draw the map and send to file
gmap3.draw("../data/chain_map.html") 
print("Done")

In [32]:
counter = 0
for chain in chains:
    counter += 1
    print(counter)
    chain.plot_pollution()

1
2
